In [2]:
import numpy as np
import os, glob
import pyfits
import dicom
import matplotlib.pyplot as plt
import tensorflow as tf
import tflearn
import sklearn, sklearn.preprocessing
%matplotlib inline

hdf5 is not supported on this machine (please install/reinstall h5py for optimal experience)


## Import and preprocessing

In [80]:
def importHeartData(calmFile, stressFile):
    """
    Import heart data and extract the pixel array.
    Concatenate and return stress file and calm file.
    """
    calmTmp = dicom.read_file(calmFile).pixel_array
    stressTmp = dicom.read_file(stressFile).pixel_array
    
    calmTmp = cropHeart(calmTmp)
    stressTmp = cropHeart(stressTmp)
    
    calmPadded = []
    stressPadded = []
    
    aC = [30-i for i in calmTmp.shape]
    aS = [30-i for i in stressTmp.shape]
    
    for i in np.arange(calmTmp.shape[0]):
        calmPadded.append(np.lib.pad(calmTmp[i], \
                                     ((aC[1]/2, aC[1]/2 + aC[1]%2), (aC[2]/2, aC[2]/2 + aC[2]%2))\
                                     'linear_ramp'))
        stressPadded.append(np.lib.pad(stressTmp[i], \
                                       ((aS[1]/2, aS[1]/2 + aS[1]%2), (aS[2]/2, aS[2]/2 + aS[2]%2))\
                                       'linear_ramp'))
    
    
    #catOut = np.concatenate([calmTmp, stressTmp])
    #return calmTmp, stressTmp

def importDir(parentDir):
    """
    Scan though directories in parent directory; look for dirs labelled 
    STRESS* or REST* in the imediate subsirs and import any dcm files in them.
    Return a dataFile of the concatenated stress and calm *.dcm files.
    """
    tmplst = []
    for dirs in os.listdir(parentDir):
        cwdStress = glob.glob(parentDir+"/"+dirs+"/STRESS*/*.dcm")
        cwdCalm = glob.glob(parentDir+"/"+dirs+"/REST*/*.dcm")
        tmplst.append(importHeartData(cwdCalm[0], cwdStress[0]))
    dataFile = np.array(tmplst)
    return dataFile

def cropHeart(inp):
    """
    Crop the heart so that all the padding is done away with.
    Output cropped heart.
    """
    # argwhere will give you the coordinates of every non-zero point
    true_points = np.argwhere(inp)
    # take the smallest points and use them as the top left of your crop
    top_left = true_points.min(axis=0)
    # take the largest points and use them as the bottom right of your crop
    bottom_right = true_points.max(axis=0)
    out = inp[top_left[0]:bottom_right[0]+1,  # plus 1 because slice isn't
          top_left[1]:bottom_right[1]+1,   # inclusive
          top_left[2]:bottom_right[2]+1]  
    print(inp.shape, out.shape)
    return out

SyntaxError: invalid syntax (<ipython-input-80-b386ced26a42>, line 19)

In [78]:
# Do data import
normDir = "./data/nlst"
normDat = importDir(normDir)
abDir = "./data/rlst"
abDat = importDir(abDir)
inData = np.concatenate([normDat[:abDat.shape[0]], abDat]) # Normal and abnormal data same shape

# Do labelling
normLab = np.zeros(normDat.shape[0])[:abDat.shape[0]]
abLab = np.ones(abDat.shape[0])
labels = np.concatenate([normLab, abLab])
    
# Mutual shuffle
shufData, shufLab = sklearn.utils.shuffle(inData, labels, random_state=1)
shufData = np.reshape(shufData,(-1,40,64,64,1))
shufLabOH = np.eye(2)[labels.astype(int)] # One hot encode

((33, 64, 64), (20, 16, 21))
((35, 64, 64), (21, 17, 22))


TypeError: pad() takes exactly 3 arguments (2 given)

## Neural net

In [ ]:
sess = tf.InteractiveSession()
tf.reset_default_graph()
tflearn.initializations.normal()

# Input layer:
net = tflearn.layers.core.input_data(shape=[None, 58, 64, 64, 1])

# First layer:
net = tflearn.layers.conv.conv_3d(net, 8, [5,5,5],  activation="leaky_relu")
net = tflearn.layers.conv.max_pool_3d(net, 2, strides=2)

# Second layer:
net = tflearn.layers.conv.conv_3d(net, 16, [5,5,5], activation="leaky_relu")
net = tflearn.layers.conv.max_pool_3d(net, 2, strides=2)

# Fully connected layer
net = tflearn.layers.core.fully_connected(net, 1024, regularizer="L2", weight_decay=0.001, activation="leaky_relu")
net = tflearn.layers.core.fully_connected(net, 1024, regularizer="L2", weight_decay=0.001, activation="leaky_relu")

# Dropout layer:
net = tflearn.layers.core.dropout(net, keep_prob=0.5)

# Output layer:
net = tflearn.layers.core.fully_connected(net, 2, activation="softmax")

net = tflearn.layers.estimator.regression(net, optimizer='adam', learning_rate=0.0001, loss='categorical_crossentropy')

model = tflearn.DNN(net, tensorboard_verbose=0)
model.fit(shufData, shufLabOH, batch_size=29, n_epoch=10, show_metric=True)

Training Step: 7  | total loss: 10.71379 | time: 84.875s
| Adam | epoch: 004 | loss: 10.71379 - acc: 0.5341 -- iter: 29/58
